In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import os
from skimage import io
from skimage.color import rgb2gray
from sklearn.utils import shuffle
from skimage.transform import rescale, resize

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer,MultiLabelBinarizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [5]:
from keras.metrics import categorical_accuracy,binary_accuracy,sparse_categorical_accuracy
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, ReLU, BatchNormalization, GlobalAveragePooling2D,LeakyReLU
from keras.optimizers import RMSprop, SGD, Adadelta, Adam, Adagrad, Adamax, Nadam
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19

Using TensorFlow backend.


In [6]:
def preprocess_image(image_path,size1,size2):

    im = io.imread(image_path)
    im = resize(im, (size1, size2,3), anti_aliasing=True)
    img = im.astype(float)
    return np.array(img)

In [7]:
def load_imagens(size1,size2):

    todas_figuras = []
    classe=[]
    Path = 'Images/'
    
    figuras=os.listdir(Path)
    for tipo in figuras:
        figura=os.listdir(Path+tipo)
        for figuras in figura:
            todas_figuras.append(preprocess_image(Path+tipo+'/'+figuras,size1,size2))
            classe.append(tipo)
            
    tamanho_pinturas = len(todas_figuras)
    tamanho_nomes = len(classe)
    print("Tamanho do arranjo figuras = ",tamanho_pinturas)
    print("Tamanho do arranjo nomes = ",tamanho_nomes)
    
    return np.array(todas_figuras),np.array(classe)

In [114]:
def createModel(input_shape, nclasses):

    model = Sequential()

    model.add(Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=input_shape,name='convo1_bloco1'))
    model.add(BatchNormalization(name='batch_normalization1_bloco1'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding="same",name='conv2_bloco1'))
    model.add(BatchNormalization(name='batch_normalization2_bloco1'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2),padding='valid',name='maxpool_bloco1'))
    
    model.add(Conv2D(128, (3, 3), activation='relu', padding="same",name='convo1_bloco2'))
    model.add(BatchNormalization(name='batch_normalization1_bloco2'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding="same",name='convo2_bloco2'))
    model.add(BatchNormalization(name='batch_normalization2_bloco2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2),padding='valid',name='maxpool_bloco2'))
    
    model.add(Conv2D(256, (3, 3), activation='relu', padding="same",name='convo1_bloco3'))
    model.add(BatchNormalization(name='batch_normalization1_bloco3'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding="same",name='convo2_bloco3'))
    model.add(BatchNormalization(name='batch_normalization2_bloco3'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding="same",name='convo3_bloco3'))
    model.add(BatchNormalization(name='batch_normalization3_bloco3'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding="same",name='convo4_bloco3'))
    model.add(BatchNormalization(name='batch_normalization4_bloco3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2),padding='valid',name='maxpool_bloco3'))
    
    model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo1_bloco4'))
    model.add(BatchNormalization(name='batch_normalization1_bloco4'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo2_bloco4'))
    model.add(BatchNormalization(name='batch_normalization2_bloco4'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo3_bloco4'))
    model.add(BatchNormalization(name='batch_normalization3_bloco4'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo4_bloco4'))
    model.add(BatchNormalization(name='batch_normalization4_bloco4'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2),padding='valid',name='maxpool_bloco4'))
    
    #model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo1_bloco5'))
    #model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo2_bloco5'))
    #model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo3_bloco5'))
    #model.add(Conv2D(512, (3, 3), activation='relu', padding="same",name='convo4_bloco5'))
    #model.add(MaxPooling2D((2, 2), strides=(2, 2),padding='valid',name='maxpool_bloco5'))
    
    model.add(Flatten(name='Flatten'))

    model.add(Dense(512, activation='relu',name='Dense_1'))
    model.add(BatchNormalization(name='batch_normalization_Dense1'))
    
    model.add(Dense(512, activation='relu',name='Dense_2'))
    model.add(BatchNormalization(name='batch_normalization_Dense2'))
    
    if(nclasses>2):
        model.add(Dense(nclasses , activation='softmax',name='Output_Dense'))
    else:
        model.add(Dense(1 , activation='sigmoid',name='Output_Dense'))

    return model

In [9]:
size1 = 64
size2 = 64

In [10]:
print("Carregando imagens")
imagens, classes = load_imagens(size1,size2)

Carregando imagens
Tamanho do arranjo figuras =  657
Tamanho do arranjo nomes =  657


In [11]:
classes = classes.reshape(-1, 1)

In [12]:
uniques, ids = np.unique(classes, return_inverse=True)

In [13]:
n_classes = len(uniques)

In [14]:
encoder =  LabelEncoder()
y_classes = encoder.fit_transform(classes)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(imagens, y_classes, test_size=0.20,random_state=42)

In [115]:
modelo = createModel(imagens[0].shape, nclasses=n_classes)
modelo.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convo1_bloco1 (Conv2D)       (None, 64, 64, 64)        1792      
_________________________________________________________________
batch_normalization1_bloco1  (None, 64, 64, 64)        256       
_________________________________________________________________
conv2_bloco1 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
batch_normalization2_bloco1  (None, 64, 64, 64)        256       
_________________________________________________________________
maxpool_bloco1 (MaxPooling2D (None, 32, 32, 64)        0         
_________________________________________________________________
convo1_bloco2 (Conv2D)       (None, 32, 32, 128)       73856     
_________________________________________________________________
batch_normalization1_bloco2  (None, 32, 32, 128)     

In [116]:
epochs = 500
batch_size = 50

In [117]:
opt = 'SGD'

In [118]:
if(opt == 'SGD'):
    print("Optimizer = SGD")
    optimizer = SGD(lr=0.1, decay=1e-2/epochs, momentum=0.95, nesterov=True)
elif (opt == 'RMS'):
    print("Optimizer = RMS")
    optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-05, decay=0.0)
elif (opt == 'Adadelta'):
    print("Optimizer = Adadelta")
    optimizer = Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
elif (opt == 'Adam'):
    print("Optimizer = Adam")
    optimizer = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-2, decay=1e-5/epochs, amsgrad=True)
elif (opt == 'Adagrad'):
    print("Optimizer = Adagrad")
    optimizer = Adagrad(lr=0.01, epsilon=None, decay=0.0)
elif (opt == 'Adamax'):
    print("Optimizer = Adamax")
    optimizer = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
elif (opt == 'NAdam'):
    print("Optimizer = NAdam")
    optimizer = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

Optimizer = SGD


In [119]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, mode='auto', epsilon=0.0001)

In [120]:
filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max',save_weights_only=True)

In [121]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [122]:
loss1='categorical_crossentropy'
loss2='sparse_categorical_crossentropy'
loss3 = "binary_crossentropy"

In [123]:
modelo.compile(optimizer=optimizer, loss=loss3, metrics=["accuracy"])

In [ ]:
history = modelo.fit(X_train, y_train, batch_size=batch_size,epochs=epochs,validation_data=(X_test, y_test), callbacks=[learning_rate_reduction,checkpoint,es])

Train on 525 samples, validate on 132 samples
Epoch 1/500
525/525 [==============================] - 134s 254ms/step - loss: 1.5635 - accuracy: 0.5886 - val_loss: 20206264568.2424 - val_accuracy: 0.4697

Epoch 00001: val_accuracy improved from -inf to 0.46970, saving model to weights.hdf5
Epoch 2/500
525/525 [==============================] - 138s 263ms/step - loss: 1.8801 - accuracy: 0.5886 - val_loss: 65822984.1818 - val_accuracy: 0.4545

Epoch 00002: val_accuracy did not improve from 0.46970
Epoch 3/500
525/525 [==============================] - 140s 267ms/step - loss: 1.1205 - accuracy: 0.6514 - val_loss: 88425724.6061 - val_accuracy: 0.4697

Epoch 00003: val_accuracy did not improve from 0.46970
Epoch 4/500
525/525 [==============================] - 119s 226ms/step - loss: 0.9645 - accuracy: 0.7086 - val_loss: 518046.7273 - val_accuracy: 0.4621

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.05000000074505806.

Epoch 00004: val_accuracy did not improve from 0.46970
Epo

525/525 [==============================] - 113s 215ms/step - loss: 0.2387 - accuracy: 0.9010 - val_loss: 0.4355 - val_accuracy: 0.8333

Epoch 00036: val_accuracy did not improve from 0.83333
Epoch 37/500
525/525 [==============================] - 113s 216ms/step - loss: 0.2183 - accuracy: 0.9086 - val_loss: 0.4272 - val_accuracy: 0.8409

Epoch 00037: val_accuracy improved from 0.83333 to 0.84091, saving model to weights.hdf5
Epoch 38/500
525/525 [==============================] - 112s 213ms/step - loss: 0.2122 - accuracy: 0.9124 - val_loss: 0.4223 - val_accuracy: 0.8333

Epoch 00038: val_accuracy did not improve from 0.84091
Epoch 39/500
525/525 [==============================] - 113s 215ms/step - loss: 0.2128 - accuracy: 0.9124 - val_loss: 0.4196 - val_accuracy: 0.8333

Epoch 00039: val_accuracy did not improve from 0.84091
Epoch 40/500
525/525 [==============================] - 127s 242ms/step - loss: 0.2388 - accuracy: 0.8990 - val_loss: 0.4152 - val_accuracy: 0.8182

Epoch 00040: R